In [1]:
# import filtering  
# df = filtering.moses_to_df("data/en-si/Ubuntu.en-si.en", "data/en-si/Ubuntu.en-si.si", "english", "sinhala")
# e1 = filtering.to_multilingual_embedding("english", df["english"], "labse")
# e2 = filtering.to_multilingual_embedding("sinhala", df["sinhala"], "labse")
# ss = filtering.find_similarity_score(e1, e2)

In [2]:
import bucc_style_dataset as bsd

In [3]:
si_mono_lines = []
eng_mono_lines = []
with open("data/en-si/sin_wikipedia_2021_30K-sentences.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.removesuffix("\n")
        split  =line.split("\t")
        si_mono_lines.append(split[1])
with open("data/en-si/eng_news_2024_30K-sentences.txt", "r", encoding="utf-8") as f:
    for line in f: 
        line = line.removesuffix("\n")
        split  =line.split("\t")
        eng_mono_lines.append(split[1])


In [4]:
si_par_lines = []
eng_par_lines = []
with open("data/en-si/eng_Latn.dev", "r", encoding="utf-8") as f:
    for line in f:
        line = line.removesuffix("\n")
        eng_par_lines.append(line)
with open("data/en-si/sin_Sinh.dev", "r", encoding="utf-8") as f:
    for line in f: 
        line = line.removesuffix("\n")
        si_par_lines.append(line)

In [5]:
train_list, test_list = bsd.split_shuffle_create_corpus(eng_mono_lines, si_mono_lines, eng_par_lines, si_par_lines)

In [6]:
en_list = train_list[0].split("\n")
si_list = train_list[1].split("\n")
en_corpus_lines = []
si_corpus_lines = []
for line in en_list:
    line = line.split("\t")[1]
    en_corpus_lines.append(line)
for line in si_list:
    line = line.split("\t")[1]
    si_corpus_lines.append(line)
import filtering
e1 = filtering.to_multilingual_embedding("english", en_corpus_lines, "labse")
e2 = filtering.to_multilingual_embedding("sinhala", si_corpus_lines, "labse")    

/home/tanmay/miniconda3/envs/thesis_wsl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
source_lines = [f"{e1.shape[0]} {e1.shape[1]}"]
target_lines = [f"{e2.shape[0]} {e2.shape[1]}"]
for sent, encoding in zip(en_corpus_lines, e1):
    sent = sent.replace(" ", "_")
    encoding_str = " ".join([f"{x:.4f}" for x in encoding])
    source_lines.append(f"{sent} {encoding_str}")
for sent, encoding in zip(si_corpus_lines, e2):
    sent = sent.replace(" ", "_")
    encoding_str = " ".join([f"{x:.4f}" for x in encoding])
    target_lines.append(f"{sent} {encoding_str}")

In [8]:
with open("source.vec", "w", encoding="utf-8") as f: 
    for line in source_lines:
        f.write(f"{line}\n")
with open("target.vec", "w", encoding="utf-8") as f: 
    for line in target_lines:
        f.write(f"{line}\n")


In [9]:
import bilingual_nearest_neighbor as bnn 
bnn.main(source_embeddings="evaluation/en.source.vec", target_embeddings="evaluation/si.target.vec", output="evaluation/en-si.output.txt", binary=0)

In [10]:
gold_file_dict = {}
with open("data/en-si/eng_Latn.dev", "r", encoding="utf-8") as f1, open("data/en-si/sin_Sinh.dev", "r", encoding="utf-8") as f2:
    for line1, line2 in zip(f1, f2):
        line1 = line1.removesuffix("\n")
        line2 = line2.removesuffix("\n")
        line1 = line1.replace("\u200d", "")
        line2 = line2.replace("\u200d", "")
        gold_file_dict[line1] = line2

In [11]:
with open("evaluation/en-si.output.txt", "r", encoding="utf-8") as f:
    test_dict = {}
    for line in f:
        split = line.split("\t")
        split = [item.replace("_", " ") for item in split]
        split = [item.replace("\u200d", "") for item in split]
        split = split[:2]
        if split[0] in gold_file_dict.keys():
            test_dict[split[0]] = split[1]

In [12]:
with open("evaluation/mined.en", "w", encoding="utf-8") as f1, open("evaluation/mined.si", "w", encoding="utf-8") as f2:
    for s1, s2 in test_dict.items():
        f1.write(s1 + "\n")
        f2.write(s2 + "\n")

In [13]:
import filtering 
filtering.main(files=["evaluation/mined.en", "evaluation/mined.si"], langs=["english", "sinhala"], output="evaluation/filtered.en-si.tsv", model="labse")

100%|██████████| 156/156 [00:00<00:00, 572450.94it/s]
2025-05-17 10:47:38,159 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: xlm-roberta-base
100%|██████████| 156/156 [00:21<00:00,  7.33it/s]
2025-05-17 10:48:01,832 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: xlm-roberta-base
100%|██████████| 156/156 [00:00<00:00, 638976.00it/s]

Raw corpus: 249

After dropping duplicates: 249

After removing length based outliers: 247

After performing language identification: 247

After filtering based on similarity scores: 156

After filtering based on word alignment: 149



In [14]:
import pandas as pd
output_df = pd.read_csv("evaluation/filtered.en-si.tsv", sep="\t")

In [15]:
correct = 0
wrong = 0
for sentence in output_df["english"]:
    if sentence in gold_file_dict: 
        if output_df.loc[output_df["english"] == sentence, "sinhala"].iloc[0] == gold_file_dict[sentence]:
            correct = correct + 1
        else: 
            wrong = wrong + 1 
print(f"{correct}\n")
print(f"{wrong}\n") 

149

0

